<a href="https://colab.research.google.com/github/weepingwillow2001/data_analysis_practice/blob/main/7%EC%9E%A5/7_5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [3]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]  # 첫 5개 표시

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

In [4]:
bins = pd.qcut(draws, 4) #  draws를 "4"분위수로 나눈다
bins
# [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] <
                                           (0.687, 3.211]]

In [7]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3',"Q4"])
# ['Q1' < 'Q2' < 'Q3' < 'Q4']
# labels 객체에 3개만 있다면 -> 오류
bins

['Q1', 'Q4', 'Q1', 'Q2', 'Q2', ..., 'Q3', 'Q3', 'Q2', 'Q3', 'Q2']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [8]:
bins.codes[:10]  # 처음 10개 코드 확인
#   codes
# Q1: 0
# Q2: 1
# Q3: 2
# Q4: 3

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

In [9]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

<ipython-input-9-e24f63e0c221>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(bins)


,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


In [10]:
results['quartile']

,quartile
0,Q1
1,Q2
2,Q3
3,Q4


In [11]:
N = 10_000_000  # Python에서 _는 큰 숫자를 읽기 쉽게 구분하는 역할을 합니다(실제 값에는 영향 없음).
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))  # 정수 나눗셈으로, 10,000,000을 4로 나눈 값인 2,500,000을 얻습니다.

In [12]:
categories = labels.astype('category')

In [ ]:
# deep=True:

# Python 객체가 실제로 차지하는 "모든" 메모리를 계산합니다.
# "문자열 같은 객체의 실제 크기를 포함"합니다.
# 더 정확하지만 계산하는 데 시간이 더 오래 걸립니다.

# deep=False:

# Series의 "기본 구조만" 계산합니다.
# 문자열, 리스트 같은 Python 객체의 실제 크기는 무시합니다.
# NumPy 배열 기반 데이터 형식(int, float 등)에는 정확하지만, 객체 데이터 형식(문자열 등)에는 부정확합니다.

In [14]:
labels.memory_usage(deep=True)    # 80000128
# 메모리에서 실제로 차지하는 바이트 크기를 반환

600000132

In [15]:
categories.memory_usage(deep=True)  # 10000540

# 카테고리 저장 방식: 카테고리 타입으로 변환하면(labels.astype('category')), pandas는:

# 고유한 값('foo', 'bar', 'baz', 'qux')을 "한 번만" 저장합니다.
# 각 위치에 실제 문자열 대신 이 "고유 값들의 인덱스(0, 1, 2, 3)"를 저장합니다.
# 이 인덱스는 "정수"이므로 문자열보다 "훨씬 적은" 메모리를 차지합니다.

10000544

In [16]:
# %time: 코드 실행 시간을 측정

%time _ = labels.astype('category')  # 범주형으로 변환하고, 결과를 _ 변수에 할당. _는 보통 결과가 필요하지 않을 때 사용하는 관례적인 변수명!

# 변환 작업에 소요된 시간을 출력

CPU times: user 504 ms, sys: 248 ms, total: 752 ms
Wall time: 750 ms


In [17]:
# %timeit: 지정된 코드를 여러 번 "반복 실행"하고(기본적으로 수천 번) 그 "평균" 실행 시간을 측정(time iterable)

%timeit labels.value_counts()       # labels.value_counts()의 평균 실행 시간을 출력
%timeit categories.value_counts()

545 ms ± 8.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
53.9 ms ± 6.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
